In [7]:
import os
import shutil
import glob

In [3]:
path="/content/drive/MyDrive/ObjectDetection-YOLOtiny"
os.chdir(path)

In [4]:
!ls

images	Untitled0.ipynb


In [5]:
# !git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15530, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 15530 (delta 6), reused 10 (delta 4), pack-reused 15514
Receiving objects: 100% (15530/15530), 14.22 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (10418/10418), done.
Updating files: 100% (2058/2058), done.


In [23]:
all_txts=glob.glob("images/*.txt")

In [10]:
# os.remove("images/classes.txt")

In [14]:
TRAIN_RATIO=0.8
VAL_RATIO=0.2

In [24]:
train_txts=all_txts[:int(len(all_txts)*TRAIN_RATIO)]
test_txts=all_txts[int(len(all_txts)*TRAIN_RATIO):]

In [19]:
os.makedirs("data/train")
os.makedirs("data/val")

In [20]:
!ls

darknet  data  images  YOLO-V4-tiny.ipynb


In [ ]:
[
    (shutil.move(txt,"data/train"),shutil.move(
        os.path.join(os.path.split(txt)[0],
                     os.path.splitext(
                        os.path.split(txt)[-1]
                        )[0]+".jpg"),"data/train"))
    for txt in train_txts
]

In [ ]:
[
    (shutil.move(txt,"data/val"),shutil.move(
        os.path.join(os.path.split(txt)[0],
                     os.path.splitext(
                        os.path.split(txt)[-1]
                        )[0]+".jpg"),"data/val"))
    for txt in test_txts
]

In [ ]:
cfg_path="/content/drive/MyDrive/ObjectDetection-YOLOtiny/darknet/cfg"
[ os.remove(os.path.join(cfg_path,cfg)) for cfg in  os.listdir(cfg_path) if ("yolov4-tiny" not in cfg) and os.path.isfile(os.path.join(cfg_path,cfg))]

In [39]:
os.listdir(cfg_path)

['yolov4-tiny-3l.cfg',
 'yolov4-tiny-custom.cfg',
 'yolov4-tiny.cfg',
 'yolov4-tiny_contrastive.cfg']

In [38]:
shutil.rmtree(os.path.join(cfg_path,"yolov1"))

In [41]:
0.8*6000

4800.0

Changes to do in yolov4 architecture as per your dataset Open darknet folder -> cfg folder -> yolov4-tiny.cfg (Delete everything else)

change batch=1 from batch=64

change subdivisions=1 from subdivisions=16

width=608 #416 height=608 #416

Next is max_batches.This parameter is very very important.The thing to remember here is that minimum batch value would be 6000 ( if you have 1 class or 2 classes or 3 classes.)

#### In our case we have 4 classes. So max_batches would be 4*2000 = 8000 i.e 8000 would be the max_batches.

#### Steps are also important. There are 2 steps:Ist step should be 90% of the max_batch value. And 2nd step would be the 80% of the max_batch
#### steps are steps=7200,6400  (90% of 8000 and 80% of 8000)


Steps are also important. There are 2 steps:Ist step should be 90% of the max_batch value. And 2nd step would be the 80% of the max_batch steps are steps=5400,4800 (90% of 6000 and 80% of 6000)

We have 3 YOLO layers for 3 different scales.

Showing you a demo below. Just see the [Yolo] layer. Above [YOLO] layer, we have [convolutional] layer. Change the number of filters as per your output classes as per this calculation (classes+5)3 = (2+5)3 = 21

In [ ]:
dark_data="darknet/data"
[os.remove(os.path.join(dark_data,f))for f in os.listdir(dark_data) if os.path.isfile(os.path.join(dark_data,f))]

In [46]:
os.listdir(dark_data)

['images', 'labels']

In [53]:
shutil.move("data",dark_data)

'darknet/data/data'

In [82]:
# os.chdir("..")

In [87]:
with open(os.path.join("data","train.txt"),mode="w") as f:
  f.writelines([f+"\n" for f in glob.glob("data/data/train/*.jpg")])
with open(os.path.join("data","test.txt"),mode="w") as f:
  f.writelines([f+"\n" for f in glob.glob("data/data/val/*.jpg")])

In [60]:
with open(os.path.join(dark_data,"obj.names"),mode="w") as f:
  f.writelines(["american\n","indian\n"])

In [62]:
with open(os.path.join(dark_data,"obj.data"),mode="w") as f:
  f.writelines([
      "classes - 2\n",
      "train - /darknet/data/train.txt\n",
      "valid - /darknet/data/test.txt\n",
      "names - /darknet/data/obj.names\n",
      "backup - /darknet/data\n"
  ])

In [63]:
os.chdir("darknet")

In [88]:
os.getcwd()

'/content/drive/MyDrive/ObjectDetection-YOLOtiny/darknet'

In [64]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [65]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_op

In [107]:
!chmod +x darknet

In [99]:
# os.chdir("..")

In [108]:
!./darknet detector train data/obj.data cfg/yolov4-tiny-custom.cfg data/yolov4-tiny.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.419352, iou_loss = 0.000000, total_loss = 0.419352 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.023898), count: 1, class_loss = 499.808167, iou_loss = 0.006409, total_loss = 499.814575 
 total_bbox = 13487, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.334670, iou_loss = 0.000000, total_loss = 0.334670 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.265449), count: 3, class_loss = 501.129730, iou_loss = 1.977844, total_loss = 503.107574 
 total_bbox = 13490, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.000000), count: 1, class_loss = 0.502367, iou_loss = 0.000000, total_loss = 0.502367 
v

In [106]:
os.getcwd()

'/content/drive/MyDrive/ObjectDetection-YOLOtiny/darknet'

In [97]:
# os.chdir("data")


# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

In [73]:
os.getcwd()

'/content/drive/MyDrive/ObjectDetection-YOLOtiny/darknet'